# Implementing character-level LSTM text generation

In [1]:
# Download and parse text file
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))


Corpus length: 600901


In [2]:
# vectorizing sequences of characters

#Max Length of character sequence
maxlen = 60

# Sample a new sequence for every three characters
step = 3

# holds our extracted sequences, empty list
sentences = []

# holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

# Dictionary  to map unique characters to their index in the list "chars"
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')

# One-hot encodes the characters into binary arrays

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200281
Unique characters: 59
Vectorization...


# Building the network

In [3]:
# Single-layer LSTM model for next-character prediction
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [4]:
# Model compilation configuration
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Training the Language Model and sampling

In [5]:
# Function to sample the next character given the model's predictions

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
#Text-generation loop
import random
import sys

#Train the model for 2 epochs
for epoch in range(1, 3):
    print('epoch', epoch)
    # Fits the model for one iteration on the data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Selects a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
   #Tries a range of different sampling temperatures
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # Generate 400 characters, starting from the seed text
        for i in range(400):
            #Onehot encoes the characters generated so far
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            
            #Samples the next character
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1565/1565 [==============================] - 252s 158ms/step - loss: 2.2589
--- Generating with seed: "e entire movement of the world
stopped short, and an all kno"
------ temperature: 0.2
e entire movement of the world
stopped short, and an all knowledge of the such the subled and the stand the has a soul and the soul and the suble in the have the contence of the soul and the soun in the conterned in the such in the himself with the soul and and the desire of the has the some and the conterned and that the such in the such the conterned the soul and the such the become that the such a some of the subled and the conternt in the such a thing 
------ temperature: 0.5
h a some of the subled and the conternt in the such a thing that is a very more it as him its contentivence of the humper that conternal what conceases to the deal the concerticies of the back and a man listlence of saticious perists the contrined and things of the one is experience of the disclusion, and superition,